# Data Discovery: Exploration and Transformation of <i>Raw Data</i>

This notebook orchestrates the preprocessing of raw data. It reads the raw data, explores it, applies transformations, and generates the processed dataset.

<b>TODO</b> Adapt this section as needed. Add more details and context where appropriate.<br>

# 1. Environment setup

## 1.1. Import libraries

# 2. Load raw data

# 3. Explore data

# 4. Define transformations